# RESNET - 50

In [2]:
import cv2
import keras
import numpy as np
from keras.datasets import cifar10, cifar100
from keras.models import Model
from keras.layers import Input, ZeroPadding2D, Dense, Dropout, Activation, Add, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, AveragePooling2D
from keras.initializers import glorot_uniform
from keras import optimizers
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [1]:
c10_class_name = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
c100_class_name = ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 
                   'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 
                   'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 
                   'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse','mushroom', 'oak_tree', 'orange', 'orchid', 
                   'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 
                   'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 
                   'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', 
                   'willow_tree', 'wolf', 'woman', 'worm']

In [3]:
def res_block(X, f, filters, stage, block, increase = False, s = 2):
    
    # Defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters   # Retrieving filters
    X_shortcut = X         # Saving input value
    stride = (1,1)
    
    if increase:
        stride = (s,s)
        
        #Shortcut Path
        X_shortcut = Conv2D(F3, (1,1), strides = (s,s), name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed = 0))(X_shortcut)
        X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = stride, padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters  = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)
    
    return X

In [4]:
def ResNet50(input_shape, classes):
    
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0), padding = 'same')(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = res_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1, increase = True)
    X = res_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = res_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3 (≈4 lines)
    X = res_block(X, f = 3, filters = [128,128,512], stage = 3, block = 'a', s = 2, increase = True)
    X = res_block(X, 3, [128,128,512], stage = 3, block = 'b')
    X = res_block(X, 3, [128,128,512], stage = 3, block = 'c')
    X = res_block(X, 3, [128,128,512], stage = 3, block = 'd')
    
    # Stage 4 (≈6 lines)
    X = res_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block = 'a', s = 2, increase = True)
    X = res_block(X, 3, [256, 256, 1024], stage = 4, block = 'b')
    X = res_block(X, 3, [256, 256, 1024], stage = 4, block = 'c')
    X = res_block(X, 3, [256, 256, 1024], stage = 4, block = 'd')
    X = res_block(X, 3, [256, 256, 1024], stage = 4, block = 'e')
    X = res_block(X, 3, [256, 256, 1024], stage = 4, block = 'f')
    
    # Stage 5 (≈3 lines)
    X = res_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block = 'a', s = 2, increase = True)
    X = res_block(X, 3, [512, 512, 2048], stage = 5, block = 'b')
    X = res_block(X, 3, [512, 512, 2048], stage = 5, block = 'c')
    
    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D((2,2))(X)
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='output', kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    resnet = Model(inputs = X_input, outputs = X, name='ResNet50')
    opt = keras.optimizers.Adam(learning_rate=0.0001)
    resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return resnet

### CIFAR - 10

In [5]:
(c10_x_train_orig, c10_y_train_orig), (c10_x_test_orig, c10_y_test_orig) = cifar10.load_data()

# Pre-processing X
c10_x_train = c10_x_train_orig.astype('float32')
c10_x_test = c10_x_test_orig.astype('float32')
c10_x_train = c10_x_train/255
c10_x_test = c10_x_test/255

# Hyper-parameters
c10_epochs = 50
c10_batch_size = 128
c10_num_classes = 10
c10_input_shape = c10_x_train.shape[1:]

# Pre-processing Y
c10_y_train = keras.utils.to_categorical(c10_y_train_orig, c10_num_classes)
c10_y_test = keras.utils.to_categorical(c10_y_test_orig, c10_num_classes)

print(c10_x_train.shape, c10_x_test.shape, c10_y_train.shape, c10_y_test.shape)

(50000, 32, 32, 3) (10000, 32, 32, 3) (50000, 10) (10000, 10)


In [6]:
c10_resnet = ResNet50(c10_input_shape, c10_num_classes)
c10_resnet.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 19, 19, 64)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 19, 19, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

In [7]:
c10_resnet.fit(c10_x_train, c10_y_train, validation_data = (c10_x_test, c10_y_test), epochs = c10_epochs, batch_size = c10_batch_size)

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 97s 2ms/step - loss: 1.7069 - accuracy: 0.4115 - val_loss: 1.9851 - val_accuracy: 0.3352
Epoch 2/50
50000/50000 [==============================] - 63s 1ms/step - loss: 1.2622 - accuracy: 0.5557 - val_loss: 1.4642 - val_accuracy: 0.4882
Epoch 3/50
50000/50000 [==============================] - 63s 1ms/step - loss: 1.0274 - accuracy: 0.6414 - val_loss: 1.7364 - val_accuracy: 0.5004
Epoch 4/50
50000/50000 [==============================] - 64s 1ms/step - loss: 0.8881 - accuracy: 0.6915 - val_loss: 1.8400 - val_accuracy: 0.4774
Epoch 5/50
50000/50000 [==============================] - 63s 1ms/step - loss: 0.7863 - accuracy: 0.7283 - val_loss: 2.6636 - val_accuracy: 0.3134
Epoch 6/50
50000/50000 [==============================] - 64s 1ms/step - loss: 0.9112 - accuracy: 0.6873 - val_loss: 1.0513 - val_accuracy: 0.6655
Epoch 7/50
50000/50000 [==============================] - 63s 1ms/st

In [8]:
scores = c10_resnet.evaluate(c10_x_test, c10_y_test)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 7s 669us/step
Test loss: 1.3620490513324737
Test accuracy: 0.7573999762535095


In [9]:
predictions = {}
for i in range(c10_num_classes):
    index = np.where(c10_y_test_orig == i)
    slice_x_test = c10_x_test[index[0]]
    slice_y_test = c10_y_test[index[0]]
    scores = c10_resnet.evaluate(slice_x_test, slice_y_test, verbose = 0)
    print('Class: ' + c10_class_name[i] + ' Test accuracy: ' + str(scores[1]))
    predictions[c10_class_name[i]] = str(scores[1])

Class: airplane Test accuracy: 0.7409999966621399
Class: automobile Test accuracy: 0.8259999752044678
Class: bird Test accuracy: 0.6420000195503235
Class: cat Test accuracy: 0.5920000076293945
Class: deer Test accuracy: 0.7649999856948853
Class: dog Test accuracy: 0.6940000057220459
Class: frog Test accuracy: 0.734000027179718
Class: horse Test accuracy: 0.7929999828338623
Class: ship Test accuracy: 0.906000018119812
Class: truck Test accuracy: 0.8809999823570251


### CIFAR - 100

In [5]:
# Load data
(c100_x_train_orig, c100_y_train_orig), (c100_x_test_orig, c100_y_test_orig) = cifar100.load_data()

# Pre-processing X
c100_x_train = c100_x_train_orig.astype('float32')
c100_x_test = c100_x_test_orig.astype('float32')
c100_x_train = c100_x_train/255
c100_x_test = c100_x_test/255

# Hyper-parameters
c100_epochs = 50
c100_batch_size = 128
c100_num_classes = 100
c100_input_shape = c100_x_train.shape[1:]

# Pre-processing Y
c100_y_train = keras.utils.to_categorical(c100_y_train_orig, c100_num_classes)
c100_y_test = keras.utils.to_categorical(c100_y_test_orig, c100_num_classes)

print(c100_x_train.shape, c100_x_test.shape, c100_y_train.shape, c100_y_test.shape)

(50000, 32, 32, 3) (10000, 32, 32, 3) (50000, 100) (10000, 100)


In [6]:
c100_resnet = ResNet50(c100_input_shape, c100_num_classes)
#c100_resnet.summary()

In [7]:
c100_resnet.fit(c100_x_train, c100_y_train, validation_data = (c100_x_test, c100_y_test), epochs = c100_epochs, batch_size = c100_batch_size)

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 97s 2ms/step - loss: 4.5406 - accuracy: 0.0612 - val_loss: 4.4104 - val_accuracy: 0.0655
Epoch 2/50
50000/50000 [==============================] - 64s 1ms/step - loss: 3.6298 - accuracy: 0.1529 - val_loss: 3.4031 - val_accuracy: 0.1817
Epoch 3/50
50000/50000 [==============================] - 64s 1ms/step - loss: 3.3080 - accuracy: 0.2114 - val_loss: 266.3710 - val_accuracy: 0.0350
Epoch 4/50
50000/50000 [==============================] - 64s 1ms/step - loss: 3.0678 - accuracy: 0.2489 - val_loss: 3.8075 - val_accuracy: 0.2409
Epoch 5/50
50000/50000 [==============================] - 64s 1ms/step - loss: 2.8159 - accuracy: 0.3005 - val_loss: 7.4542 - val_accuracy: 0.1833
Epoch 6/50
50000/50000 [==============================] - 64s 1ms/step - loss: 3.1372 - accuracy: 0.2392 - val_loss: 3.5761 - val_accuracy: 0.2206
Epoch 7/50
50000/50000 [==============================] - 64s 1ms/

In [8]:
scores = c100_resnet.evaluate(c100_x_test, c100_y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 7s 650us/step
Test loss: 4.5791174369812015
Test accuracy: 0.400299996137619


In [9]:
predictions = {}
for i in range(c100_num_classes):
    index = np.where(c100_y_test_orig == i)
    slice_x_test = c100_x_test[index[0]]
    slice_y_test = c100_y_test[index[0]]
    scores = c100_resnet.evaluate(slice_x_test, slice_y_test, verbose = 0)
    print('Class: ' + c100_class_name[i] + ' Test accuracy: ' + str(scores[1]))
    predictions[c100_class_name[i]] = str(scores[1])

Class: apple Test accuracy: 0.7200000286102295
Class: aquarium_fish Test accuracy: 0.49000000953674316
Class: baby Test accuracy: 0.28999999165534973
Class: bear Test accuracy: 0.3100000023841858
Class: beaver Test accuracy: 0.20000000298023224
Class: bed Test accuracy: 0.4099999964237213
Class: bee Test accuracy: 0.4300000071525574
Class: beetle Test accuracy: 0.38999998569488525
Class: bicycle Test accuracy: 0.4099999964237213
Class: bottle Test accuracy: 0.5099999904632568
Class: bowl Test accuracy: 0.30000001192092896
Class: boy Test accuracy: 0.28999999165534973
Class: bridge Test accuracy: 0.44999998807907104
Class: bus Test accuracy: 0.20000000298023224
Class: butterfly Test accuracy: 0.20999999344348907
Class: camel Test accuracy: 0.3199999928474426
Class: can Test accuracy: 0.36000001430511475
Class: castle Test accuracy: 0.5600000023841858
Class: caterpillar Test accuracy: 0.30000001192092896
Class: cattle Test accuracy: 0.1599999964237213
Class: chair Test accuracy: 0.670000

In [10]:
predictions

{'apple': '0.7200000286102295',
 'aquarium_fish': '0.49000000953674316',
 'baby': '0.28999999165534973',
 'bear': '0.3100000023841858',
 'beaver': '0.20000000298023224',
 'bed': '0.4099999964237213',
 'bee': '0.4300000071525574',
 'beetle': '0.38999998569488525',
 'bicycle': '0.4099999964237213',
 'bottle': '0.5099999904632568',
 'bowl': '0.30000001192092896',
 'boy': '0.28999999165534973',
 'bridge': '0.44999998807907104',
 'bus': '0.20000000298023224',
 'butterfly': '0.20999999344348907',
 'camel': '0.3199999928474426',
 'can': '0.36000001430511475',
 'castle': '0.5600000023841858',
 'caterpillar': '0.30000001192092896',
 'cattle': '0.1599999964237213',
 'chair': '0.6700000166893005',
 'chimpanzee': '0.6499999761581421',
 'clock': '0.38999998569488525',
 'cloud': '0.5',
 'cockroach': '0.6299999952316284',
 'couch': '0.36000001430511475',
 'crab': '0.2800000011920929',
 'crocodile': '0.36000001430511475',
 'cup': '0.6600000262260437',
 'dinosaur': '0.3400000035762787',
 'dolphin': '0.